# AEA Tutorial
In this tutorial we will walk through the steps to access [Aria Everyday Activities (AEA) dataset](link) The AEA dataset contains:
* Raw data from 1-2 [Project Aria glasses](https://projectaria.com)
    * Data is synchronized where recordings are made with two Aria glasses in the same location
* Speech to Text annotation
* Machine Perception Services [(MPS)](https://facebookresearch.github.io/projectaria_tools/docs/ARK/mps) derived data:
    * General Eye Gaze 
    * Trajectory 
        * Open loop trajectories
        * Closed loop trajectories
        * Calibration data
    * 3D Point Clouds
  
This tutorial covers how to:
* Access raw sensor data (VRS files)
* Visualize Eye Gaze data
* Visualize Speech data
* Load concurrent sequences from multiple Project Aria glasses in a shared space location
* Use Timecode to get synchronized timestamps 


In [ ]:
# Specifics for Google Colab
google_colab_env = 'google.colab' in str(get_ipython())
if google_colab_env:
    print("Running from Google Colab, installing projectaria_tools and getting sample data")
    !pip install projectaria-tools>=1.3.3

In [ ]:
import os
import subprocess
from projectaria_tools.projects.aea import (
    AriaEverydayActivitiesDataPathsProvider, 
    AriaEverydayActivitiesDataProvider)
import numpy as np

import matplotlib.pyplot as plt

from projectaria_tools.core import calibration, mps
from projectaria_tools.core.sensor_data import TimeDomain, TimeQueryOptions
from projectaria_tools.core.mps import get_eyegaze_point_at_depth

from projectaria_tools.core.sophus import SE3
from projectaria_tools.core.stream_id import StreamId

## Create AEA data provider
The AEA data provider is similar to the [Project Aria Tools data provider](https://facebookresearch.github.io/projectaria_tools/docs/data_utilities/core_code_snippets/data_provider), with the additional ability to access speech data and data from multiple concurrent recordings. 

* Create AEA data provider and access all the following providers
    * mps provider
    * speech provider
    * vrs provider
* Create AEA data_path_provider and access the specific file directory
    * Get sequence meta data
    * Get paths of vrs, mps, and speech
    * Get path of concurrent recording

In [ ]:
if google_colab_env:
    aea_sample_path = "./aea_sample_data"
else:
    aea_sample_path = "/tmp/aea_sample_data"

data_sequence_url_rec1 = "https://www.projectaria.com/async/sample/download/?bucket=aea&filename=loc1_script2_seq1_rec1_10s_sample.zip"
data_sequence_url_rec2 = "https://www.projectaria.com/async/sample/download/?bucket=aea&filename=loc1_script2_seq1_rec2_10s_sample.zip"

sequence_name_rec1 = "loc1_script2_seq1_rec1_10s_sample"
sequence_name_rec2 = "loc1_script2_seq1_rec2_10s_sample"

sequence_path_rec1 = f"{aea_sample_path}/{sequence_name_rec1}"
sequence_path_rec2 = f"{aea_sample_path}/{sequence_name_rec2}"

command_list = [
    f"mkdir -p {aea_sample_path}",
    f"mkdir -p {sequence_path_rec1}",
    f"mkdir -p {sequence_path_rec2}",
    # Download sample data
    f'curl -o {aea_sample_path}/aea_sample_data_rec1.zip -C - -O -L "{data_sequence_url_rec1}"',
    f'curl -o {aea_sample_path}/aea_sample_data_rec2.zip -C - -O -L "{data_sequence_url_rec2}"',
    # Unzip the sample data
    f"unzip -o {aea_sample_path}/aea_sample_data_rec1.zip -d {sequence_path_rec1}",
    f"unzip -o {aea_sample_path}/aea_sample_data_rec2.zip -d {sequence_path_rec2}",

]

# Execute the commands for downloading dataset
if google_colab_env:
    for command in command_list:
        !$command
else:
    for command in command_list:
        subprocess.run(command, shell=True, check=True)

In [ ]:
# Create AEA path provider
aea_paths_provider_1 = AriaEverydayActivitiesDataPathsProvider(sequence_path_rec1)

# create AEA data provider
aea_data_provider_1 = AriaEverydayActivitiesDataProvider(sequence_path_rec1)

## Show AEA data paths
AEA data path from a sequence can be obtained using AEA path provider

In [ ]:
print(aea_paths_provider_1.get_data_paths())

## Access concurrent recordings
Query metadata.json to find and load the directory of any concurrent recording into the AEA data_path_provider.
If your recording doesn’t have a concurrent recording you’ll get the error: “RuntimeError: Timedomain TimeCode not supported by stream RGB Camera Class #1”.


In [ ]:
# Get concurrent recording filename and load into AEA data provider
concurrent_recording_filename = aea_paths_provider_1.get_concurrent_recordings()
concurrent_sequence_path = os.path.join(os.path.dirname(sequence_path_rec1), concurrent_recording_filename[0])

# Load concurrent sequence directly
aea_data_provider_2 = AriaEverydayActivitiesDataProvider(concurrent_sequence_path)

## Access synchronized timestamp from both provider
Timestamp can come from four different places:
* TimeDomain.DEVICE_TIME
* TimeDomain.TIME_CODE
* TimeDomain.RECORD_TIME
* TimeDomain.HOST_TIME

**TimeDomain.TIME_CODE** is commonly used to register timestamp of multiple synchronized recordings.

For AEA dataset, TimeDomain.TIME_CODE is used first to find match and TimeDomain.DeviceTime is used to query data from MPS and speech.

In [ ]:
RGB_STREAM_ID = StreamId("214-1")

# timecode timestamps for recording 1
timecode_vec_1 = aea_data_provider_1.vrs.get_timestamps_ns(RGB_STREAM_ID, TimeDomain.TIME_CODE)

# timecode timestamps for recording 2
timecode_vec_2 = aea_data_provider_2.vrs.get_timestamps_ns(RGB_STREAM_ID, TimeDomain.TIME_CODE)

# Convert the common timecode timestamp to unique DeviceTime timestamp for each sequence
for timecode_time_ns in timecode_vec_1:
        device_time_ns_1 = aea_data_provider_1.vrs.convert_from_timecode_to_device_time_ns(timecode_time_ns)
        device_time_ns_2 = aea_data_provider_2.vrs.convert_from_timecode_to_device_time_ns(timecode_time_ns)
        print(f"TimeCode {timecode_time_ns} converts to DeviceTime of sequence 1: {device_time_ns_1} and sequence 2: {device_time_ns_2}")

## Access speech data
Obtain speech data provider directly from aea_data_provider and query the speech data by sentence or words with timestamp in nanoseconds.
* Speech data can be only be queried with `TimeDomain.DEVICE_TIME`
* Timestamp conversion from / to `TimeDomain.TIME_CODE` using `convert_from_timecode_to_device_time_ns`
* Obtain sentence/words from speech data

In [ ]:
timecode_timestamp = timecode_vec_1[100]
device_time_ns_1 = aea_data_provider_1.vrs.convert_from_timecode_to_device_time_ns(timecode_time_ns)
device_time_ns_2 = aea_data_provider_2.vrs.convert_from_timecode_to_device_time_ns(timecode_time_ns)

In [ ]:
# obtain sentence at query_time_ns
sentence = aea_data_provider_1.speech.get_sentence_data_by_timestamp_ns(device_time_ns_1, TimeQueryOptions.CLOSEST)
print(f"Sentence from sequence 1 at device timestamp {device_time_ns_1} is : '{sentence}'")

# obtain word at query_time_ns
sentence = aea_data_provider_2.speech.get_sentence_data_by_timestamp_ns(device_time_ns_2, TimeQueryOptions.CLOSEST)
print(f"Sentence from sequence 2 at device timestamp {device_time_ns_2} is : '{sentence}'")

## Visualize RGB image with Gaze

In [ ]:
def draw_eye_gaze(aea_data_provider : AriaEverydayActivitiesDataProvider, device_time_ns: int, depth_m: float):
    rgb_stream_label = aea_data_provider.vrs.get_label_from_stream_id(RGB_STREAM_ID)
    device_calibration = aea_data_provider.vrs.get_device_calibration()
    rgb_camera_calibration = device_calibration.get_camera_calib(rgb_stream_label)
    
    image = aea_data_provider.vrs.get_image_data_by_time_ns(
            RGB_STREAM_ID, device_time_ns, TimeDomain.DEVICE_TIME, TimeQueryOptions.BEFORE)    
    eye_gaze = aea_data_provider.mps.get_general_eyegaze(device_time_ns, TimeQueryOptions.CLOSEST)
    
    assert eye_gaze, "Eye gaze not available"
    # Compute eye_gaze vector at depth_m reprojection in the image
    gaze_vector_in_cpf = mps.get_eyegaze_point_at_depth(
        eye_gaze.yaw, eye_gaze.pitch, depth_m
    )
    T_device_CPF = device_calibration.get_transform_device_cpf()
    gaze_center_in_camera = (
        rgb_camera_calibration.get_transform_device_camera().inverse()
        @ T_device_CPF
        @ gaze_vector_in_cpf
    )
    gaze_projection = rgb_camera_calibration.project(gaze_center_in_camera)
    
    # Project the gaze center in rgb camera frame
    if gaze_projection is not None:
        fig, ax = plt.subplots()
        ax.imshow(image[0].to_numpy_array());
        ax.axis('off')
        
        # Plot the cross
        u, v = gaze_projection.flatten()
        plt.plot(u, v, '+', c="red", mew=1, ms=20); plt.xticks([]); plt.yticks([]); 
        plt.show()
    else:
        print(f"Eye gaze center projected to {gaze_projection}, which is out of camera sensor plane.")

In [ ]:
# draw general eye gaze with synchronized device timestamp at depth = 1.0 meter
eye_gaze_depth = 1.0 # meters
draw_eye_gaze(aea_data_provider_1, device_time_ns_1, eye_gaze_depth)
draw_eye_gaze(aea_data_provider_2, device_time_ns_2, eye_gaze_depth)